In [ ]:
import numpy as np
import pandas as pd
import os
import struct
from sklearn.datasets import fetch_mldata
import matplotlib.pyplot as plt

path = os.path.join(os.getcwd(), 'datasets')
def load_mnist(path, kind):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels.idx1-ubyte'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images.idx3-ubyte'
                               % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)
 
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)
 
    return images, labels
train_images, train_labels = load_mnist(path, 'train')
test_images, test_labels = load_mnist(path, 't10k')


'''显示测试集中的一幅图像'''
some_data = train_images[40000]
some_data_image = some_data.reshape(28, 28)
plt.imshow(some_data_image, cmap=plt.cm.binary, interpolation='nearest')  # interpolation差值
plt.axis('off')  # 隐藏坐标轴
plt.show()


'''一些模型对数据的排序敏感，因此在训练之前先将数据的顺序打乱'''
shuffle_train_index = np.random.permutation(60000)  # permutation排序
x_train, y_train = train_images[shuffle_train_index], train_labels[shuffle_train_index]
shuffle_test_index = np.random.permutation(10000)
x_test, y_test = test_images[shuffle_test_index], test_labels[shuffle_test_index]

'''首先现简化一下问题,只尝试去识别一个数字,比如说,数字 5。这个“数字 5 检测器”就是
一个二分类器,能够识别两类别,“是 5”和“非 5”。'''
#创建目标向量
y_train_5 = (y_train==5)
y_test_5 = (y_test==5)

'''随机梯度下降分类器SGD，这个分类器有一个好处是能够高效地处理非常大的数据
集。这部分原因在于SGD一次只处理一条数据,这也使得 SGD 适合在线学习'''
from sklearn.linear_model.stochastic_gradient import SGDClassifier
#max_iter指的是最大进行1000次迭代，若要采用minibatch，则可使用parcial_fit而不是fit  
#tol  当loss > previous_loss - tol(previous_loss=loss时，即损失保持基本不变)时，停止迭代
sgd_clf = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3, n_jobs=-1)
sgd_clf.fit(x_train, y_train_5)
'''交叉验证'''
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, x_train, y_train_5, cv=3, scoring='accuracy')  # 交叉验证，k值为3

'''以下使用精度，PR曲线，ROC曲线三种方法对分类器的性能进行评估'''
'''设计一个非常蠢的分类器，有 90% 的精度。这是因为只有 10% 的图片是数字 5,所以你总是
猜测某张图片不是 5,你也会有90%的可能性是对的。当其中一些类比其他类频繁得多，精度不是一个非常准确的指标'''
# from sklearn.base import BaseEstimator
# class Never5classfier(BaseEstimator):  # 感觉是继承
#     def fit(self, X, y=None):   # 直接不进行拟合
#         pass
#     def predict(self, X):  # 直接返回一个0矩阵
#         return np.zeros((len(X), 1), dtype=bool)  # (len(X), 1)为形状，表示返回一个X行1列的0向量
# never_5_clf = Never5classfier()
# never_5_clf.fit(x_train, y_train_5)
# cross_val_score(never_5_clf, x_train, y_train_5, cv=3, scoring='accuracy')

from sklearn.model_selection import cross_val_predict
#基于交叉验证得到的一组测试值，3折以其他2折作为训练集可预测另外一折的值，并保留。同样的方法可以得到另外两折的值，最后合并输出
y_train_pre = cross_val_predict(sgd_clf, x_train, y_train_5, cv=3,)

In [ ]:
'''使用混淆矩阵进行模型评估'''
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pre)
'''计算查准率(准确率)和查全率(召回率)'''
from sklearn.metrics import precision_score, recall_score, f1_score
#说明了一张5的图片识别为5的概率为73%，而且之检测出‘5’图片的82%
precision_score(y_train_5, y_train_pre) # 输出0.7323..
recall_score(y_train_5, y_train_pre) # 输出0.8256..
f1_score(y_train_5, y_train_pre) # f1值

'''不同的项目对查准率和查全率的倾向不同，如根据视频信息查找罪犯，需要较高的查全率'''
'''SGDClassifier对于每个样例,它根据决策函数计算分数,如果这个分数大于一个阈值,它会将样例分配给正例,
否则它将分配给反例。通过调整thresholds,选出好的准确率/召回率折衷的方法'''
#通过set method为decision_function返回训练集的分数
y_score = cross_val_predict(sgd_clf, x_train, y_train_5, cv=3, method='decision_function', n_jobs=10)
from sklearn.metrics import precision_recall_curve
#通过precision_recall_curve返回thresholds变化时的precision and recall，thresholds 大于它为5小于它为非5
#然后绘制图像观察recall and precision
precision, recall, thresholds = precision_recall_curve(y_train_5, y_score)
def plot_precision_recall_vs_threshold(precision, recall, thresholds):
    plt.figure()
    plt.plot(thresholds, precision[:-1], 'k--', label='precision')
    plt.plot(thresholds, recall[:-1], c='red', label='recall')
    plt.xlabel('thresholds')
    plt.legend(loc='best')
    plt.xlim([-10000, 10000])
    plt.ylim([0, 1])
    plt.show()
plot_precision_recall_vs_threshold(precision, recall, thresholds)
# '''若想要90%的准确率，可以用下面的方法找到一个阈值，使得准确率达到90%'''
y_train_pred_90 = (y_score > 5000)
print(precision_score(y_train_5, y_train_pred_90))
print(recall_score(y_train_5, y_train_pred_90))  

'''PR曲线'''
'''选出好的准确率/召回率折衷的方法是直接画出准确率(查准率)对召回率(查全率)的曲线
根据实际项目情况进行选择'''
def plot_precision_vs_recall():
    plt.figure(num=2)  # figure和show函数最好放在函数外避免多次调用。这里不做修改了
    plt.plot(recall[:-1], precision[:-1])
    plt.xlabel('recall')
    plt.ylabel('precision')
    plt.xlim([0,1])
    plt.ylim([0,1])   
    plt.show()
plot_precision_vs_recall()

In [ ]:
'''绘制ROC曲线，在正例较少时PR曲线更优'''
'''一个好的分类器的 ROC 曲线应该尽可能向左上角方向靠拢'''
'''可以通过比较ROC曲线下的面积(AUC)比较分类器的性能，一个完美的分类器的 AUC 等于 1'''
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_train_5, y_score)
def plot_roc(fpr, tpr, label=None, linestyle=None):
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, 'red', label=label, linestyle=linestyle)
    plt.axis([0, 1, 0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc = 'best')
plt.figure()
plot_roc(fpr, tpr, label='sgd')
plt.show()

In [ ]:
'''通过比较AUC来判断两个分类器的性能'''
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train_5, y_score)
#加入随机森林分类器进行比较,回执roc曲线需要的是样例的分数，一个简单的方法是使用正例的概率当做样例的分数
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42, n_estimators=10, n_jobs=10)
'''predict_proba()方法返回一个数组,数组的每一行代表一个样例,
每一列代表一个类。数组当中的值的意思是:给定一个样例属于给定类的概率。
比如,70%的概率这幅图是数字 5。此例中输出两列，猜测为第一列表示非5的概率,第二列表示为5概率'''
y_probas_forest = cross_val_predict(forest_clf, x_train, y_train_5, cv=3, method='predict_proba')
y_score_forest = y_probas_forest[:, 1] # 因为要绘制ROC曲线，需要的是样例的分数，以，为5的概率作为分数 
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5, y_score_forest)
'''绘制两个分类器的ROC曲线比较性能'''
plt.figure()
plot_roc(fpr_forest, tpr_forest, label="forest", linestyle="--")
plot_roc(fpr, tpr, label="sgd", linestyle='-')
plt.show()
roc_auc_score(y_train_5, y_score_forest)  # 随机森林分类器AUC

In [ ]:
'''多类分类'''
'''现在让我们检测更多的数字,而不仅仅是一个数字 5'''
'''一些算法(比如随机森林分类器或者朴素贝叶斯分类器)可以直接处理多类分类问题。其他
一些算法(比如 SVM 分类器或者线性分类器)则是严格的二分类器。然后,有许多策略可以
让你用二分类器去执行多类分类。
“一对所有”(OvA)策略：训练10个二分类器,每一个对应一个数字(探测器 0,探测器 1,探测器 2,以此类推)
“一对一”(OvO)策略：一个分类器用来处理数字 0 和数字 1,一个用来处理数字 0 和数字 2,一个用来处理数字 1 和 2,以此类推。'''
'''OvO 策略的主要优点是:每个分类器只需要在训练集的部分数据上面进行训
练。这部分数据是它所需要区分的那两个类对应的数据。如果有 N 个类。你需要训练N*(N-1)/2个分类器
一些算法(比如 SVM 分类器)在训练集的大小上很难扩展,所以对于这些算法,OvO 是比
较好的,因为它可以在小的数据集上面可以更多地训练,较之于巨大的数据集而言。但是,
对于大部分的二分类器来说,OvA 是更好的选择。'''
'''Scikit-Learn 可以探测出你想使用一个二分类器去完成多分类的任务,它会自动地执行OvA'''
sgd_clf.fit(x_train, y_train)  # 对所有数据进行训练可以预测所有类，实际上在幕后执行了OVA策略
sgd_clf.predict([some_data]) 
some_data_score = sgd_clf.decision_function([some_data]) # 返回每个类(0-9)的分数
sgd_clf.classes_  # 返回目标类别
print(np.argmax(some_data_score)) #返回为7,返回分数最大的下标

'''基于sgd分类器的OVO策略'''
from sklearn.multiclass import OneVsOneClassifier
ovo_clf = OneVsOneClassifier(sgd_clf)
ovo_clf.fit(x_train, y_train)
ovo_clf.predict([some_data])
len(ovo_clf.estimators_)  # 返回ovo策略中训练的分类器的个数

'''训练随机森林分类器'''
forest_clf.fit(x_train, y_train)
forest_clf.predict([some_data])
forest_clf.predict_proba([some_data])  # 返回样例 是每个类型的 概率
'''利用交叉验证评估多分类器'''
cross_val_score(sgd_clf, x_train, y_train, cv=3, scoring='accuracy', n_jobs=10)
'''可以利用对数据正则化来提升分类器的精度'''
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#x_train.astype拷贝x_train并将他的数据转换为制定类型(np.float64)
#fit_transform先进行数据拟合，在进行标准化
x_train_scaler = scaler.fit_transform(x_train.astype(np.float64))
cross_val_score(sgd_clf, x_train_scaler, y_train, cv=3, scoring='accuracy', n_jobs=10)
'''多分类混淆矩阵'''
y_train_pred = cross_val_predict(sgd_clf, x_train_scaler, y_train, cv=3)
#使用交叉验证的预测值进行评估可以更好的验证模型的泛化能力吧
conf_mx = confusion_matrix(y_train, y_train_pred)

In [ ]:
plt.matshow(conf_mx, cmap=plt.cm.gray) #将混淆矩阵以图像的方式呈现
plt.show()
row_sum = conf_mx.sum(axis=1, keepdims=True)  # keepdim保持维度,每行求和
norm_conf_mx = conf_mx / row_sum  # 将混淆矩阵的每一个值除以相应类别的图片的总数目，可得到错误率
np.fill_diagonal(norm_conf_mx, 0)  # 主对角线变为0,即不管分类正确的
#index为真值，column为预测值
'''分析混淆矩阵通常可以给你提供深刻的见解去改善你的分类器。回顾这幅图,看样子你应该
努力改善分类器在数字 8 和数字 9 上的表现,和纠正 3/5 的混淆。举例子,你可以尝试去收
集更多的数据,或者你可以构造新的、有助于分类器的特征。举例子,写一个算法去数闭合
的环(比如,数字 8 有两个环,数字 6 有一个, 5 没有)。又或者你可以预处理图片(比
如,使用 Scikit-Learn,Pillow, OpenCV)去构造一个模式,比如闭合的环。'''
plt.matshow(norm_conf_mx, cmap=plt.cm.gray) # 错误率越高，则亮度越高
plt.show()

'''显示错误分类的图片进行观察'''
cl_a, cl_b = 3, 5
X_aa = x_train[(y_train == cl_a) & (y_train_pred == cl_a)] 
X_ab = x_train[(y_train == cl_a) & (y_train_pred == cl_b)] # 被错误分类为5的3的图片
X_ba = x_train[(y_train == cl_b) & (y_train_pred == cl_a)] # 被错误分类为3的5的图片
X_bb = x_train[(y_train == cl_b) & (y_train_pred == cl_b)] 
# ax1 = plt.figure(figsize=())
# for i in range(25):
#     plt.subplot(5, 5, i+1); plt.imshow(X_aa[i].reshape(28, 28), cmap=plt.cm.binary)
#     plt.axis('off')
# plt.show()

two_dem = X_aa[:25].reshape((25, 28, 28))
arr1 = np.c_[two_dem[0], two_dem[1], two_dem[2], two_dem[3], two_dem[4]]
arr2 = np.c_[two_dem[5], two_dem[6], two_dem[7], two_dem[8], two_dem[9]]
arr3 = np.r_[arr1, arr2]

plt.figure(figsize=(8,8))
plt.subplot(221); 
plt.imshow(arr3, cmap=plt.cm.binary)
plt.subplot(222); 
plt.subplot(223); 
plt.subplot(224); 
plt.show()
'''3 和 5 之间的主要差异是连接顶部的线和底部的线的细线的位置。如果你画一个 3,连接处稍
微向左偏移,分类器很可能将它分类成 5。反之亦然。换一个说法,这个分类器对于图片的位
移和旋转相当敏感。所以,减轻 3/5 混淆的一个方法是对图片进行预处理,确保它们都很好
地中心化和不过度旋转。这同样很可能帮助减轻其他类型的错误。'''
'''多标签分类P103, 书中举了一个例子，利用y_train_large = y_train >= 7得到y_train大于7的一组布尔量标签，
利用y_train_odd = (y_train % 2 == 1)得到y_train中为奇数的一组布尔量标签，利用np.c_合并y_train_large
和y_train_odd得到一组标签，将标签作为label，y_train作为样例输入分类器，训练好后，就可以输出x是否大于7和是
否为奇数的一组标签，神奇。输入一张图片，观察图片是否具有某个特征（标签），有则为1。'''
'''多输出分类P104, 书中举了一个例子，利用noise = rnd.randint(0, 100, (len(X_train), 784))，
X_train_mod = X_train + noise，对图像添加噪声，以不带有噪声的图片作为label，带有噪声的图片作为样例输入分
类器，经过训练之后，输入一张带有噪声的图片，可输出一张干净的图片，神奇。注意到这个分类器的输出是多标签的(一个像素一个标签)和每个标签可以有多个值
(像素强度取值范围从 0 到 255)。所以它是一个多输出分类系统的例子。而多标签分类中，标签的值为0或者1'''